In [ ]:
import sys
import gpytorch
import numpy as np
import pandas as pd
import torch
from datetime import datetime
from scipy.stats import qmc
import itertools
from itertools import combinations_with_replacement, combinations, permutations

import bo_methods_lib
from bo_methods_lib.bo_methods_lib.GPBO_Classes_New import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Class_fxns import * #Fix this later
from bo_methods_lib.bo_methods_lib.analyze_data import * #Fix this later
from bo_methods_lib.bo_methods_lib.GPBO_Classes_plotters import * #Fix this later
import pympler
import pickle
import signac
from sklearn.preprocessing import RobustScaler

from pympler import asizeof

from matplotlib import pyplot as plt

import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 200
from textwrap import fill

#Ignore inconcistent version warning
import warnings
# from sklearn.exceptions import InconsistentVersionWarning
# warnings.filterwarnings(action='ignore', category=InconsistentVersionWarning)
import tensorflow_probability as tfp
warnings.filterwarnings('ignore')

In [ ]:
#Make Simulator and Training Data
cs_name_val = 3
noise_mean = 0
noise_std = None
seed = 1
#Define method, ep_enum classes, indecies to consider, and kernel
meth_name = Method_name_enum(3)
method = GPBO_Methods(meth_name)
gen_meth_theta = Gen_meth_enum(1)
gen_meth_x = Gen_meth_enum(2)
num_x_data = 5

#Define Simulator Class (Export your Simulator Object Here)
simulator = simulator_helper_test_fxns(cs_name_val, noise_mean, noise_std, seed)
num_theta_data = len(simulator.indeces_to_consider)*10
#Generate Exp Data (OR Add your own experimental data as a Data class object)
set_seed = 1 #Set set_seed to 1 for data generation
gen_meth_x = Gen_meth_enum(gen_meth_x)
exp_data = simulator.gen_exp_data(num_x_data, gen_meth_x, set_seed)
#Set simulator noise_std artifically as 5% of y_exp mean (So that noise will be set rather than trained)
simulator.noise_std = np.abs(np.mean(exp_data.y_vals))*0.05
print(simulator.noise_std)
#Note at present, training data is always the same between jobs since we set the data generation seed to 1
sim_data = simulator.gen_sim_data(num_theta_data, num_x_data, gen_meth_theta, gen_meth_x, 1.0, seed, False)
val_data = simulator.gen_sim_data(10, 10, Gen_meth_enum(1), Gen_meth_enum(1), 1.0, seed + 1, False)
# print(sim_data.theta_vals)

In [ ]:
if method.emulator == True:
    all_gp_data = sim_data
    all_val_data = val_data
else:
    all_gp_data = simulator.sim_data_to_sse_sim_data(method, sim_data, exp_data, 1.0, False)
    print(len(all_gp_data.theta_vals), len(all_gp_data.x_vals), len(all_gp_data.y_vals))
    all_val_data = simulator.sim_data_to_sse_sim_data(method, val_data, exp_data, 1.0, False)
    print(len(all_val_data.theta_vals), len(all_val_data.x_vals), len(all_val_data.y_vals))

In [ ]:
#Scale training data if necessary
scalerY = RobustScaler(unit_variance = True)
# scalerY = StandardScaler()
y = scalerY.fit_transform(sim_data.y_vals.reshape(-1,1))
y_test = scalerY.transform(val_data.y_vals.reshape(-1,1))
# y = sim_data.y_vals.reshape(-1,1)

In [ ]:
plt.hist(y, bins=50, cumulative = True, density=True, stacked=True, edgecolor = 'k', label = "Train") 
plt.hist(y_test, bins=50, cumulative = True, stacked=True, density=True, label = "Test", edgecolor = 'k', alpha = 0.5) # Adjust the number of bins as needed
xmin, xmax = plt.xlim() 
mu, std = norm.fit(y[~np.isnan(y)])
x = np.linspace(xmin, xmax, 100) 
p = norm.cdf(x, mu, std) 
# plt.plot(x, p, 'g', linewidth=2, label = "Normal CDF") 

percentile_90_train = np.percentile(y[~np.isnan(y)], 95)
percentile_90_test = np.percentile(y_test[~np.isnan(y_test)], 95)
print(percentile_90_train, percentile_90_test)

val1 = -2
val2 = 2
train_between = np.sum((y >= val1) & (y <= val2))
train_proportion = train_between / len(y)

# Calculate the proportion for the testing set
test_between = np.sum((y_test >= val1) & (y_test <= val2))
test_proportion = test_between / len(y_test)

print(train_proportion, test_proportion)

# Add labels and title
plt.xlabel(r'$y^{sim}$' +' Value')
plt.ylabel('Cumulative Probability Density')
plt.title('Histogram of Müller ' + r'$y_0$' ' Data')
plt.legend(loc = "best")

plt.savefig("cs3_hist.png")